# Autoregressive (자동회귀) 문장 생성

간단한 자동 회귀적인 텍스트 생성을 위한 코드 예제입니다. 여기서는 OpenAI의 GPT-2 모델을 사용하는데, GPT-3은 API를 통해서만 직접 사용할 수 있기 때문입니다. 그럼에도 GPT-2는 GPT-3와 매우 유사한 구조를 가지고 있으므로, 동일한 접근 방식을 사용합니다.

### Colab에서 실행

In [2]:
!pip install -q transformers # create huggingface.co

In [3]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# 토크나이저 초기화
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")

In [4]:
# 문장 시작 부분
input_text = "Once upon a time"
input_ids = tokenizer.encode(input_text, return_tensors="pt")
input_ids

tensor([[7454, 2402,  257,  640]])

In [5]:
# 모델 초기화
model = GPT2LMHeadModel.from_pretrained("gpt2-medium")

In [10]:
# 문장 생성 (auto-regressive 사용)
output = model.generate(input_ids, max_length=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
output

tensor([[ 7454,  2402,   257,   640,    11,   612,   373,   257,   582,   508,
          5615,   287,   257,  7404,  1444,   509, 17716,   322,    13,   679,
           373,   257,   845,   922,   582,    11,   290,   339,   373,   845,
          1611,   284,   465,  1751,    13,  1881,  1110,    11,   339,   373,
          6155,  1863,   262,  2975,    11,   290,   339,  2497,   257,  2415]])

In [17]:
# try to apply temperature param
output = model.generate(input_ids, max_length=50, num_return_sequences=1, temperature=1, pad_token_id=tokenizer.eos_token_id)
output

tensor([[ 7454,  2402,   257,   640,    11,   612,   373,   257,   582,   508,
          5615,   287,   257,  7404,  1444,   509, 17716,   322,    13,   679,
           373,   257,   845,   922,   582,    11,   290,   339,   373,   845,
          1611,   284,   465,  1751,    13,  1881,  1110,    11,   339,   373,
          6155,  1863,   262,  2975,    11,   290,   339,  2497,   257,  2415]])

In [12]:
# try change max_length value
output = model.generate(input_ids, max_length=20, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
output

tensor([[ 7454,  2402,   257,   640,    11,   612,   373,   257,   582,   508,
          5615,   287,   257,  7404,  1444,   509, 17716,   322,    13,   679]])

In [13]:
# 생성된 문장 출력
for i, generated_text in enumerate(output):
    decoded_text = tokenizer.decode(generated_text, skip_special_tokens=True)
    print(f"Generated text {i + 1}: {decoded_text}")

Generated text 1: Once upon a time, there was a man who lived in a village called Krakow. He


GPT-2는 자체적으로 autoregressive 모델입니다. "Autoregressive"란, 이전에 생성된 토큰들을 기반으로 다음 토큰을 생성하는 모델을 의미합니다.

위의 코드에서 `model.generate` 메서드는 이미 autoregressive한 방식으로 문장을 생성합니다. 그러나 이를 명시적으로 보여주기 위해 각 단계에서 토큰을 하나씩 생성하는 autoregressive한 코드를 아래에 작성하겠습니다:

In [5]:
# 문장 시작 부분
input_text = "Once upon a time"
input_ids = tokenizer.encode(input_text, return_tensors="pt")
input_ids

tensor([[7454, 2402,  257,  640]])

input_ids를 입력으로 받아 next token의 확률 분포를 예측값으로 반환 받습니다.

In [6]:
predictions = model(input_ids)
logits = predictions.logits
logits

tensor([[[ -74.6887,  -74.2220,  -80.2354,  ...,  -83.7431,  -80.4567,
           -76.2926],
         [-225.0849, -225.3535, -229.2500,  ..., -236.9878, -237.3179,
          -224.3200],
         [  -6.1224,   -4.9656,  -10.4817,  ...,  -13.5870,  -12.2557,
            -5.6444],
         [  -9.2671,   -7.4639,  -14.5420,  ...,  -16.8138,  -16.4736,
            -9.4272]]], grad_fn=<UnsafeViewBackward0>)

예측값 중 가장 큰 값의 token을 선택합니다.

In [7]:
predicted_token = torch.argmax(logits[0, -1]).item()

print(f"예측한 next token : {predicted_token}")
token = tokenizer.decode(predicted_token, skip_special_tokens=True)
print(f"decoding한 next token : {token}")

예측한 next token : 11
decoding한 next token : ,


위와 같은 방식으로 마지막 단어 이어 붙이기 형식의 문장 생성을 합니다.

In [14]:
# Autoregressive한 방식으로 문장 생성
max_length = 20                     # 생성할 문장의 최대 길이 설정
input_ids_concat = input_ids  # 초기 입력 토큰(예: 문장의 시작 부분)

# 생성할 문장의 길이가 max_length보다 작을 동안 반복
while input_ids_concat.shape[1] < max_length:
    # 모델을 사용하여 다음 토큰 예측
    predictions = model(input_ids_concat)
    logits = predictions.logits                  # 예측된 로짓을 가져옴
    predicted_token = torch.argmax(logits[0, -1]).item()  # 로짓에서 가장 확률이 높은 토큰을 선택

    # 예측된 토큰을 기존의 입력 토큰 뒤에 추가
    input_ids_concat = torch.cat([input_ids_concat, torch.tensor([[predicted_token]])], dim=1)
    print(input_ids_concat)   # 현재까지 예측된 토큰들을 출력


tensor([[7454, 2402,  257,  640,   11]])
tensor([[7454, 2402,  257,  640,   11,  612]])
tensor([[7454, 2402,  257,  640,   11,  612,  373]])
tensor([[7454, 2402,  257,  640,   11,  612,  373,  257]])
tensor([[7454, 2402,  257,  640,   11,  612,  373,  257,  582]])
tensor([[7454, 2402,  257,  640,   11,  612,  373,  257,  582,  508]])
tensor([[7454, 2402,  257,  640,   11,  612,  373,  257,  582,  508, 5615]])
tensor([[7454, 2402,  257,  640,   11,  612,  373,  257,  582,  508, 5615,  287]])
tensor([[7454, 2402,  257,  640,   11,  612,  373,  257,  582,  508, 5615,  287,
          257]])
tensor([[7454, 2402,  257,  640,   11,  612,  373,  257,  582,  508, 5615,  287,
          257, 7404]])
tensor([[7454, 2402,  257,  640,   11,  612,  373,  257,  582,  508, 5615,  287,
          257, 7404, 1444]])
tensor([[7454, 2402,  257,  640,   11,  612,  373,  257,  582,  508, 5615,  287,
          257, 7404, 1444,  509]])
tensor([[ 7454,  2402,   257,   640,    11,   612,   373,   257,   582,   50

In [15]:
# 최종적으로 생성된 문장 출력
decoded_text = tokenizer.decode(input_ids_concat[0], skip_special_tokens=True)
print(decoded_text)  # 디코딩하여 텍스트 형태로 출력

Once upon a time, there was a man who lived in a village called Krakow. He
